- read from a local json file for annotation, transform to index
- read from url for pictures, get features with encoder model provided by the author

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# !pwd  # /content  # import vocab
!cp /content/gdrive/MyDrive/PPlusPlus/utils/build_vocab.py /content/build_vocab.py
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
import torch
from torchvision import transforms
import json
from PIL import Image
import requests
from build_vocab import Vocabulary
import nltk
import pickle
import time
nltk.download('punkt')

Mounted at /content/gdrive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
embed_size=256
hidden_size=512
num_layers=1
vocab_size=4533
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class EncoderCNN(nn.Module):
  def __init__(self, embed_size):
    """Load the pretrained ResNet-152 and replace top fc layer."""
    super(EncoderCNN, self).__init__()
    # BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    resnet = models.resnet152(weights='DEFAULT')
    modules = list(resnet.children())[:-1]      # delete the last fc layer.
    self.resnet = nn.Sequential(*modules)
    # Linear(in_features=2048, out_features=256, bias=True)
    self.linear = nn.Linear(resnet.fc.in_features, embed_size)
    # BatchNorm1d(256, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
    self.init_weights()
      
  def init_weights(self):
    """Initialize the weights."""
    self.linear.weight.data.normal_(0.0, 0.02)
    self.linear.bias.data.fill_(0)
      
  def forward(self, images):  # return from load_image() in utils/sample.py
    """Extract the image feature vectors."""
    features = self.resnet(images)  # {Tensor: (1, 2048, 1, 1)}
    features = Variable(features.data)  # {Tensor: (1, 2048, 1, 1)}
    features = features.view(features.size(0), -1)  # {Tensor: (1, 2048)}
    features = self.bn(self.linear(features))  # {Tensor: (1, 256)}
    return features

*yield* training pairs for coco 

In [ ]:
def getCocoData():
  """
  get training item in desired input format (vectors of indices) from
  'coco_data/annotations_trainval2014/captions_train2014.json'
  :return:
  features: vector after pre-trained CNN
  captions: to be used in embeddings = self.embed(captions)
  """
  cnn = EncoderCNN(embed_size)
  cnn.eval()
  # self.encoder.load_state_dict(torch.load(self.encoder_path, map_location={'cuda:0': 'cpu'}))
  cnn.load_state_dict(torch.load('/content/gdrive/MyDrive/PPlusPlus/models/coco-encoder-5-3000.pkl', map_location={'cuda:0': 'cpu'}))
  if torch.cuda.is_available():
    cnn.cuda()
  transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                          (0.229, 0.224, 0.225))])
  json_data = json.loads(open('/content/gdrive/MyDrive/PPlusPlus/coco_data/annotations_trainval2014/captions_train2014.json', 'r').read())
  with open('/content/gdrive/MyDrive/PPlusPlus/coco_data/vocab_coco_4533.pkl', 'rb') as f:
    vocab = pickle.load(f)

  for each_img in json_data['annotations']:
    # {'image_id': 318556, 'caption': 'A very clean and well decorated empty bathroom'}
    image_id = each_img['image_id']  # 318556
    # http://images.cocodataset.org/train2014/COCO_train2014_000000318556.jpg
    file_url = 'http://images.cocodataset.org/train2014/COCO_train2014_000000' \
                + str(image_id) + ".jpg"
    response = requests.get(file_url, stream=True)
    if response.status_code == 200:
      # the picture exists
      im = Image.open(requests.get(file_url, stream=True).raw)
      if im.mode != 'RGB': continue  
      # im.show()
      # resize
      im = im.resize([224, 224])
      # transform (1, 3, 224, 224)
      im = transform(im).unsqueeze(0) 
      # feed into cnn
      # self.encoder(to_var(load_image(url, self.transform))); to_var() from utils/sample.py
      if torch.cuda.is_available():
        im = im.cuda()  # {Tensor: (1, 3, 224, 224)}
      features = cnn(Variable(im))  # cnn.forward(Variable(im1))

      # read captions, get index
      phrase = each_img['caption']
      tokens = nltk.tokenize.word_tokenize(phrase.lower())
      captions = [vocab.word2idx["<start>"]]
      # + [vocab.word2idx[word] for word in tokens]]
      len_token = 0
      for word in tokens:
        if word in vocab.word2idx:
          captions.append(vocab.word2idx[word])
        else:
          captions.append(vocab.word2idx["<unk>"])
        len_token += 1  # 1 - 10
        if len_token > 9:
          break
      captions.append(vocab.word2idx["<end>"])
      # padding
      while len_token < 10:
        captions.append(vocab.word2idx["<pad>"])
        len_token += 1
      captions = torch.tensor(captions)
      yield features, captions, phrase, file_url


In [ ]:
# download resnet
for feature, captions in getCocoData():
  print(feature.shape, captions.shape)
  # print(feature, '\n', captions)
  break

true
torch.Size([1, 256]) torch.Size([12])


pickle feature and caption pairs in 40ks

In [ ]:
# yield 40k pairs and pickle append to a file
batch_temp = []  # batch container
counter = 0
last_time = time.time()
file_index = 0
for feature, captions in getCocoPair():
  batch_temp.append((feature, captions))
  counter += 1
  if counter % 40000 == 0:  ###############
    filename = 'coco_feat_cap_' + str(file_index) + ".pkl"
    file_index += 1
    with open(filename, 'wb') as fp:
      torch.save(batch_temp, fp)
    batch_temp = []  # reset the container
    print(str(counter), "\t", str((time.time() - last_time)/60), "\t", filename)
    last_time = time.time()
    # break  ############
  elif counter % 10000 == 0:  # time tracking
    print(counter, "\t", str((time.time() - last_time)/60))
    last_time = time.time()

if len(batch_temp) != 0:
  # save the last bits
  print(str(len(batch_temp)) + " pairs remaining")
  filename = 'coco_feat_cap_' + str(file_index) + ".pkl"
  with open(filename, 'wb') as fp:
      torch.save(batch_temp, fp)

print(str(counter), " pairs in total")


true
true
true
true
true
true
true
true
true
true
10 	 0.11673959096272786
true
true
true
true
true
true
true
true
true
true
20 	 0.09613877534866333
true
true
true
true
true
true
true
true
true
true
30 	 0.08899432023366292
true
true
true
true
true
true
true


KeyboardInterrupt: ignored

| ks |
| -- |
|time|

In [ ]:
#To load from pickle file
# data = []
# with open('/content/coco_feat_cap_0.pkl', 'rb') as fr:
#     try:
#         while True:
#             data.append(pickle.load(fr))
#     except EOFError:
#         pass

# print(data)

with open('/content/coco_feat_cap_0.pkl', 'rb') as fr:
  coco_data = torch.load(fr)
for i, j in coco_data:
  print(i.shape, j.shape)
  print(j)

torch.Size([1, 256]) torch.Size([12])
tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11,  2,  0,  0])
torch.Size([1, 256]) torch.Size([12])
tensor([ 1,  4, 12, 13, 14,  4, 15,  7, 16, 14, 17,  2])
torch.Size([1, 256]) torch.Size([12])
tensor([ 1,  4, 20,  7, 21, 11, 22, 23, 24, 25, 26,  2])
torch.Size([1, 256]) torch.Size([12])
tensor([ 1,  4, 12, 27, 14,  4, 15,  7, 28, 29,  2,  0])


In [ ]:
sum([1/44, 3/44, 2/44, 2/44, 4/44, 1/44, 3/44, 2/44, 5/44, 2/44, 1/44, 6/44, 1/44, 4/44, 2/44, 2/44, 1/44, 1/44, 1/44])
sum([1, 3, 2, 2, 4, 1, 3, 2, 5, 2, 1, 6, 1, 4, 2, 2, 1, 1, 1])


44